In [1]:

import Simulation.pys.para.model_para as Mp
import Simulation.pys.full_sim.Catalogue_one_shell as cata_shell
import numpy as np
from importlib import reload
import pickle as pkl

In [2]:
with open('../Model_data/test_cosmo_pack.pkl', 'rb') as f:
    cosmo_pack = pkl.load(f)

with open('../Model_data/test_dust_pack.pkl', 'rb') as f:
    dust_pack = pkl.load(f)

In [ ]:
with open('../quick_py/quick_Model_data/discrete_EBVRvmap.pkl', 'rb') as f:
    retrived_map = pkl.load(f)

In [4]:
np.ones_like(retrived_map['EBV_in_equ'])*3

In [5]:
obs_f = {"EBV_in": retrived_map['EBV_in_equ'], #np.zeros_like(EBV_in),
         "EBV_out": retrived_map['EBV_out_equ'], #np.zeros_like(EBV_out),
         "Band_coefficient_in": dust_pack['band_coefficient_Rv'],
         "Band_coefficient_out": dust_pack['band_coefficient_ccm_31'],
         "ErrorModel_clean": Mp.errModel_clean,
         "ErrorModel_obs":Mp.errModel_dust,
         "nside":Mp.nside,
         "photoz_est": [Mp.photo_z_estimator],
         "Rv_map": retrived_map['Rv_equ'],
         "mask": retrived_map['mask'] * np.array(Mp.SFD<0.2) == 1,
         "zbin_in": Mp.zbin_in,
         "zbin_out": Mp.zbin_out}



In [6]:
unique_vals, counts = np.unique(retrived_map['EBV_out_equ'], return_counts=True)
print("Values:", unique_vals)
print("Counts:", counts)

In [7]:
Ncount_Exp_shell_map = cata_shell.ExpNcount_map_shells(cosmo_pack["density_map"], Mp.Ncount_hist)

In [8]:
reload(cata_shell)
shell_id = 3
cata_one_shell = cata_shell.Catalogue_one_shell(Ncount_Exp_shell_map[shell_id], Mp.template_cata_shell[shell_id], obs_f)
cata_one_shell.Sample_photometry()

In [9]:
reload(cata_shell)
cata_one_shell.Add_extinction_info()
cata_one_shell.Add_Exintction()
cata_one_shell.Add_noise(True, True)
cata_one_shell.Dereddening()
cata_one_shell.Estimate_photo_z(choice = "dust")
cata_one_shell.Estimate_photo_z(choice = "clean")


In [10]:
cata_one_shell.sampled_catalogue_shell[["EBV_in"]].value_counts()

In [11]:
cata_one_shell.sampled_catalogue_shell[["EBV_out"]].value_counts()

In [12]:
hist_dust = cata_one_shell.Statistics_on_shell(Mp.zbin_out, "dust")
hist_clean = cata_one_shell.Statistics_on_shell(Mp.zbin_out, "clean")

In [13]:
np.shape(hist_dust)

In [14]:
mask__ =  retrived_map['mask'] & ~ np.array(Mp.SFD < 0.2)
indices = np.where(mask__)[0]

In [15]:
results = cata_shell.Simulate_shells(cosmo_pack["density_map"],Mp.Ncount_hist,Mp.template_cata_shell,
                           obs_f,Mp.zbin_out,choice = "both", expectation_total_sample=2e8)

In [17]:
with open('../Model_data/test_simulation_result_pack_discrete.pkl', 'wb') as f:
    pack = {"cosmo_pack": cosmo_pack, "dust_pack": dust_pack, "data": results, "mask":obs_f["mask"]}
    pkl.dump(pack, f)